In [1]:
import sys
import torch

sys.path.append('../CMR-AI/mmaction/')
sys.path.append('../CMR-AI/')

from abc import ABCMeta, abstractmethod
from swinTransformer3D_origin import SwinTransformer3D
from models.losses.mutil_class_loss import FocalLoss, cal_auc
from models.losses.weighted_auc_f1 import get_weighted_auc_f1
from load_dataset import ACDC
from utilsss import generate_mask_matrix, pruning_mask, row_softmax
from Policy import Policy, train_agent

import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import torch
from torchvision import transforms
import pandas as pd
from skimage import transform
import numpy as np
from torch import nn
import SimpleITK as sitk
from torch.utils.data import DataLoader
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

/public/home/chenweilin/.local/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
num_class = {
        0: ['HCM', 'RV', 'DCM', 'MINF'],
        1: ['HCM', 'RV'], 
        2: ['DCM', 'MINF'],
    }

save_models = {0: 'full', 1: '1', 2: '2'}
dummy_labels = num_class[0] # taken from paper

models = []
for _, v in num_class.items():
    num_class_ = len(v)
    models.append(SwinTransformer3D(num_class=num_class_))
print(f'Total model is {len(models)}')

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3214.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Total model is 3


In [3]:
input_size, teacher_num = 0, len(models)
for name, param in list(models[0].named_parameters())[-2:-1]:
    print(f"层名: {name} | 大小: {param.size()}")
    input_size = param.size()[-1]
agent = Policy(input_size=input_size, teacher_num=teacher_num)

层名: head.fc_cls.weight | 大小: torch.Size([4, 1024])


In [5]:
load_swin_base_patch244_window877_kinetics600_22k = False
for i in range(len(models)):
    if load_swin_base_patch244_window877_kinetics600_22k:
        _pretrained_dict = torch.load(f'./ACDC/MTRL-inner-NOKL/{save_models[i]}/epoch_200.pth') 
        pretrained_dict = {}
        for k, v in _pretrained_dict.items():
            if k.startswith('module.'):
                new_key = k.replace('module.', '')
            # 移除 'cls_head.' 前缀
            elif k.startswith('cls_head.'):
                new_key = k.replace('cls_head.', '')
            else:
                new_key = k  # 如果没有前缀，则保持不变

            pretrained_dict[new_key] = v
    else:
        print(f'load weightd from local file pth.')
        _pretrained_dict = torch.load(f'./ACDC/MTL-KD/epoch_75.pth')
        pretrained_dict = {}
        for k, v in _pretrained_dict.items():
            if k.startswith('module.'):
                new_key = k.replace('module.', '')
            # 移除 'cls_head.' 前缀
            elif k.startswith('cls_head.'):
                new_key = k.replace('cls_head.', '')
            else:
                new_key = k  # 如果没有前缀，则保持不变

            pretrained_dict[new_key] = v
    model_dict = models[i].state_dict()
    # 过滤掉不匹配的参数
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    model_dict.update(pretrained_dict)
    models[i].load_state_dict(pretrained_dict, strict=False)
    print(f'No.{i+1} model load pretrained weighted end.')

load weightd from local file pth.
No.1 model load pretrained weighted end.
load weightd from local file pth.
No.2 model load pretrained weighted end.
load weightd from local file pth.
No.3 model load pretrained weighted end.


In [6]:
train_data = pd.read_csv('./roi_processed/training/train_data.csv', encoding='GBK')
test_data = pd.read_csv('./roi_processed/testing/test_data.csv', encoding='GBK')
dummy_labels = ['HCM', 'RV', 'DCM', 'MINF']  # ['HCM', 'DCM', 'MINF', 'RV', 'NOR'] # taken from paper
train_data = train_data[train_data['Finding Labels'].isin(dummy_labels)]
test_data = test_data[test_data['Finding Labels'].isin(dummy_labels)]

In [7]:
# One Hot Encoding of Finding Labels to dummy_labels
for label in dummy_labels:
    train_data[label] = train_data['Finding Labels'].map(lambda result: 1.0 if label in result else 0)

In [8]:
# One Hot Encoding of Finding Labels to dummy_labels
for label in dummy_labels:
    test_data[label] = test_data['Finding Labels'].map(lambda result: 1.0 if label in result else 0)

In [9]:
train_data['target_vector'] = train_data.apply(lambda target: [target[dummy_labels].values], 1).map(lambda target: target[0])

test_data['target_vector'] = test_data.apply(lambda target: [target[dummy_labels].values], 1).map(lambda target: target[0])

In [10]:
clean_labels = train_data[dummy_labels].sum().sort_values(ascending= False) # get sorted value_count for clean labels
print(f'训练数据集：')
print(clean_labels) # view tabular results

训练数据集：
HCM     20.0
RV      20.0
DCM     20.0
MINF    20.0
dtype: float64


In [11]:
print(f'测试数据集：')
clean_labels = test_data[dummy_labels].sum().sort_values(ascending= False) # get sorted value_count for clean labels
print(clean_labels) # view tabular results

测试数据集：
HCM     10.0
RV      10.0
DCM     10.0
MINF    10.0
dtype: float64


## 训练开始

In [12]:
base_lr = 0.0005
batch_size = 8
max_epoch = 600

phase = 'train'
  
for i in range(len(models)):
    models[i] = models[i].cuda()
    
fn_loss  = FocalLoss(device = 'cuda:0', gamma = 2.).to('cuda:0')
kl_loss = torch.nn.KLDivLoss(reduction='batchmean')

cross_loss = nn.CrossEntropyLoss()
optimizers = []
for i in range(len(models)):
    optimizers.append(torch.optim.SGD(models[i].parameters(), lr=base_lr))
    
agent_optimizer = torch.optim.SGD(agent.parameters(), lr=base_lr)
    
train_acdc_data = ACDC(data=train_data, phase = 'train', img_size=(224, 224))
train_data_loader = DataLoader(train_acdc_data, batch_size=batch_size, shuffle=True, num_workers=5)
test_acdc_data = ACDC(data=test_data, phase = 'test', img_size=(224, 224))
test_data_loader = DataLoader(test_acdc_data, batch_size=batch_size, shuffle=True, num_workers=5)

In [13]:
from torch.utils.tensorboard import SummaryWriter
# 初始化 TensorBoard
writer = SummaryWriter(log_dir='./runs/VSTMTRL-ACDC')  # 指定日志保存路径

In [ ]:
if phase == 'train':
    import time
    pre_mutil_teacher_acc = []
    now_mutil_teacher_acc = []
    for i in range(1, len(models)):
        pre_mutil_teacher_acc.append(0)
        now_mutil_teacher_acc.append(0)
    logit_actions_list = []
    agent_rewars_list  = []
    student_infos_list = []    
    for epoch_num in range(0, max_epoch):
        print(f"--------> epoch_num: {epoch_num}")
        train_loader_nums = len(train_data_loader.dataset)
        probs = np.zeros((train_loader_nums, len(num_class[0])), dtype = np.float32)
        gt    = np.zeros((train_loader_nums, len(num_class[0])), dtype = np.float32)
        k_index = 0
        start_time = time.time()
        total_train_loss = 0.0
        correct = 0.0
        for i in range(len(models)): models[i].train()
        mutil_teacher_correct = []
        mutil_teacher_num = []
        for i in range(1, len(models)):
            mutil_teacher_correct.append(0)
            mutil_teacher_num.append(0)
        ## 全局定义
        # s_train_loss = torch.tensor(0.0, device='cuda')  # 创建在GPU上的零张量
        for batch_idx, (batch_data, batch_finding, batch_label) in enumerate(train_data_loader):
            weighted_list = []
            mutil_teacher_label = torch.zeros_like(batch_label)
            pre_lables = 0
            ## 局部赋值
            s_train_loss = 0.01
            for i, k in enumerate(num_class.keys()):
                if i == 0: 
                    continue
                else:
                    ## train teacher.
                    teacher_train_data_index = pd.Series(batch_finding).isin(num_class[k])
                    teacher_train_data_index = teacher_train_data_index.to_numpy()
                    weighted_list.append(np.sum(teacher_train_data_index > 0))
                    mutil_teacher_num[i-1] += weighted_list[i-1]
                    t_train_data = batch_data[teacher_train_data_index]
                    t_train_label = batch_label[teacher_train_data_index][:, pre_lables:pre_lables+len(num_class[k])]
                    if np.sum(teacher_train_data_index > 0) == 0: 
                        pre_lables += len(num_class[i])
                        continue
                    t_output, _ = models[i](t_train_data.cuda())
                    mutil_teacher_label[teacher_train_data_index, pre_lables:pre_lables+len(num_class[k])] = \
                                            row_softmax(t_output.cpu().detach())
                    pre_lables += len(num_class[k])
                    t_output = t_output.reshape(t_output.shape[0], -1)
                    t_train_label = t_train_label.reshape(t_train_label.shape[0], -1).cuda()
                    t_train_loss = fn_loss(t_output, t_train_label)
                    optimizers[i].zero_grad()
                    t_train_loss.backward()
                    optimizers[i].step()
                    predicted_ = torch.argmax(t_output, 1)
                    labels_ = torch.argmax(t_train_label.cuda(), 1)
                    correct_ = (predicted_ == labels_).sum().item() 
                    mutil_teacher_correct[i-1] += correct_
            
            outputs, student_info = models[0](batch_data.cuda())
            outputs = outputs.reshape(outputs.shape[0], -1)
            
            mutil_teacher_label = mutil_teacher_label.reshape(mutil_teacher_label.shape[0], -1).cuda()
            probs[k_index: k_index + outputs.shape[0], :] = outputs.cpu().detach().numpy()
            gt[   k_index: k_index + outputs.shape[0], :] = batch_label.cpu().detach().numpy()
            k_index += outputs.shape[0]
            
            if (epoch_num+1) <= 100:
                log_outputs = torch.nn.LogSoftmax(dim=1)(outputs)            
                train_loss = cross_loss(outputs, batch_label.cuda()) + 0.5*kl_loss(log_outputs, mutil_teacher_label.cuda())
            else:
                 train_loss = cross_loss(outputs, batch_label.cuda())
                    
            optimizers[0].zero_grad()
            train_loss.backward()
            optimizers[0].step()
            total_train_loss += train_loss
            predicted = torch.argmax(outputs, 1)
            labels = torch.argmax(batch_label.cuda(), 1)
            correct += (predicted == labels).sum().item() 
                
        print(f"epoch_num {epoch_num} train loss {total_train_loss} ")  
        
        writer.add_scalars('Training Metrics', {
            'Loss': total_train_loss,
            'Accuracy': correct / train_loader_nums,
        }, epoch_num)
        
        lr_ = base_lr*(1-0.0009)
        for i in range(len(optimizers)):
            for param_group in optimizers[i].param_groups:
                param_group['lr'] = lr_
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"程序运行时间：{elapsed_time} 秒")
        
        save_interval = 25
        if (epoch_num + 1) % save_interval == 0:
            for i in range(len(models)):
                save_mode_path = os.path.join(f'./ACDC/MTL-KD', 'epoch_' + str(epoch_num+1) + '.pth')
                torch.save(models[i].state_dict(), save_mode_path)
                print("save model to {}".format(save_mode_path))
                
        test_interval = 25
        if (epoch_num + 1) % test_interval == 0:
            ### 模型验证
            test_loader_nums = len(test_data_loader.dataset)
            test_probs = np.zeros((test_loader_nums, len(num_class[0])), dtype = np.float32)
            test_gt    = np.zeros((test_loader_nums, len(num_class[0])), dtype = np.float32)
            test_k  =0
            models[0].eval()
            with torch.no_grad():
                for test_data_batch, _, test_label_batch in test_data_loader:
                    test_data_batch = test_data_batch.cuda()
                    test_label_batch = test_label_batch.cuda()
                    test_outputs, _ = models[0](test_data_batch.cuda())
                    test_outputs = test_outputs.reshape(test_outputs.shape[0], -1)           
                    test_label_batch = test_label_batch.reshape(test_outputs.shape[0], -1)
                    test_probs[test_k: test_k + test_outputs.shape[0], :] = test_outputs.cpu().detach().numpy()
                    test_gt[   test_k: test_k + test_outputs.shape[0], :] = test_label_batch.cpu().detach().numpy()
                    test_k += test_outputs.shape[0]
                test_label = np.argmax(test_gt, axis=1)
                test_pred = np.argmax(test_probs, axis=1)
                get_weighted_auc_f1(test_probs, test_pred, test_label)
                print(f"acc: {np.sum(test_label==test_pred)/test_loader_nums}")

In [15]:
total_weighted_acc_list = []
total_weighted_auroc_list = []
### 模型验证
for i in range(10):
    test_loader_nums = len(test_data_loader.dataset)
    test_probs = np.zeros((test_loader_nums, len(dummy_labels)), dtype = np.float32)
    test_gt    = np.zeros((test_loader_nums, len(dummy_labels)), dtype = np.float32)
    test_k  =0
    models[0].eval()
    with torch.no_grad():
        for test_data_batch, _, test_label_batch in test_data_loader:
            test_data_batch = test_data_batch.cuda()
            test_label_batch = test_label_batch.cuda()
            test_outputs, _ = models[0](test_data_batch.cuda())
            test_outputs = test_outputs.reshape(test_outputs.shape[0], -1)           
            test_label_batch = test_label_batch.reshape(test_outputs.shape[0], -1)
            test_probs[test_k: test_k + test_outputs.shape[0], :] = test_outputs.cpu().detach().numpy()
            test_gt[   test_k: test_k + test_outputs.shape[0], :] = test_label_batch.cpu().detach().numpy()
            test_k += test_outputs.shape[0]
        test_label = np.argmax(test_gt, axis=1)
        test_pred = np.argmax(test_probs, axis=1)
        weighted_auroc = get_weighted_auc_f1(test_probs, test_pred, test_label)

        cm = confusion_matrix(test_label, test_pred)
        dataset_list = [10, 10, 10, 10]  # , 7
        acc_list = []
        weighted_acc = 0.0
        for i in range(len(dataset_list)):
            weight = dataset_list[i] / sum(dataset_list)
            correct = cm[i][i]
            acc = float(correct) / dataset_list[i]
            acc_list.append(acc)
            weighted_acc += weight*acc 
        total_weighted_auroc_list.append(weighted_auroc)
        total_weighted_acc_list.append(weighted_acc)

--------------------------------------------------
auc_list : [0.23666666666666666, 0.4600000000000001, 0.87, 0.31666666666666665]
weighted_auroc:  0.4708333333333334
weighted_F1:  nan
--------------------------------------------------
auc_list : [0.29333333333333333, 0.44666666666666666, 0.7733333333333333, 0.38]
weighted_auroc:  0.4733333333333333
weighted_F1:  nan
--------------------------------------------------
auc_list : [0.24333333333333335, 0.59, 0.7633333333333333, 0.32]
weighted_auroc:  0.4791666666666667
weighted_F1:  nan
--------------------------------------------------
auc_list : [0.30333333333333334, 0.48333333333333334, 0.74, 0.29333333333333333]
weighted_auroc:  0.45499999999999996
weighted_F1:  nan
--------------------------------------------------
auc_list : [0.26666666666666666, 0.39999999999999997, 0.7433333333333333, 0.29000000000000004]
weighted_auroc:  0.425
weighted_F1:  nan
--------------------------------------------------
auc_list : [0.32, 0.503333333333333

21s   \
12619 mb